In [2]:
import os
import sys

module_path = os.path.join(os.getcwd(), '../src')
sys.path.append(module_path)

import torch 
from gan_t2i.models.CLIP import CLIPModel

from gan_t2i.models.GAN import WGAN

from gan_t2i.datasets.DatasetFactory import DatasetFactory
import torchvision.transforms as transforms
import clip
from PIL import Image

# Loading data from checkpoints 

In [ ]:
checkpoints_path = os.path.join(os.getcwd(), "checkpoints")

checkpoint = torch.load(os.path.join(checkpoints_path, "CLIPModel_epoch-2.pt"))
clip_model = CLIPModel()
clip_model.load_state_dict(checkpoint['model_state_dict'])

# Loading dataset

In [3]:
""" Text Transformation

You need to tokenize your text before passing it to the model.
"""
def tokenize_text(text):
    
    # Try except is needed due to error thrown by CLIP model that limit the context size
    # to 77 tokens so we need to split the text in smaller chunks and keep only a small portion
    # of it if the text is too long
    try:
        return clip.tokenize([text])[0]
    except:
        return clip.tokenize([text.split(".")[0]])[0]    
    

In [4]:
""" Image transformations """
transform_img = transforms.Compose([
    transforms.Resize(224, interpolation=Image.BICUBIC),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    
    # Previously computed mean and std
    transforms.Normalize([0.4355, 0.3777, 0.2879], [0.2571, 0.2028, 0.2101])
])

NameError: name 'Image' is not defined

In [ ]:
dataset = DatasetFactory.Flowers(os.path.join(os.getcwd(), "..", "data"), transform_img=transform_img, transform_caption=tokenize_text)

# Creating the GAN model and training it

In [ ]:
embedding_size = 128
p_emb_dim = 128
WGAN_model = WGAN(clip_model,embedding_size,p_emb_dim)
